<a href="https://colab.research.google.com/github/kapamawi/AI/blob/main/2_5_5___multidoc_rag_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install -qU  llama-parse llama-index-llms-openai llama-index-embeddings-openai llama-index-vector-stores-lancedb

W pierwszej linijce instalowane są cztery pakiety Pythona za pomocą narzędzia pip z następującymi flagami:
- -q oznacza tryb cichy (quiet), który ogranicza wyświetlane komunikaty
- -U wymusza aktualizację do najnowszej wersji

Instalowane pakiety to:
- llama-parse: narzędzie do parsowania różnych formatów dokumentów
- llama-index-llms-openai: integracja z modelami językowymi OpenAI
- llama-index-embeddings-openai: moduł do tworzenia embedingów tekstowych przy użyciu modeli OpenAI
- llama-index-vector-stores-lancedb: adapter do bazy danych wektorowej LanceDB

Te biblioteki razem tworzą środowisko do pracy z dokumentami i ich semantyczną analizą - parsowanie treści, zamiana tekstu na wektory i przechowywanie ich w bazie danych, a następnie możliwość wyszukiwania podobnych fragmentów przy użyciu modeli językowych.

In [ ]:
import json
import logging
import os
import time
from datetime import datetime, timedelta
from typing import Any, Dict, List, Optional

from llama_index.core import (
   Document,
   Settings,
   SimpleDirectoryReader,
   StorageContext,
   VectorStoreIndex,
)
from llama_index.core.agent import AgentRunner, FunctionCallingAgentWorker
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.tools import FunctionTool, ToolOutput
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.llms.openai import OpenAI
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from tqdm import tqdm

from google.colab import userdata


Przeanalizujmy te importy linia po linii:

Ze standardowej biblioteki Pythona importowane są:
- json: do obsługi formatu JSON
- logging: do tworzenia logów
- os: do operacji systemowych
- time: do operacji związanych z czasem
- datetime, timedelta: do operacji na datach
- typing (Any, Dict, List, Optional): do typowania w Pythonie

Z pakietu llama_index.core importowane są główne komponenty:
- Document: reprezentacja dokumentu
- Settings: ustawienia frameworka
- SimpleDirectoryReader: czytanie plików z katalogu
- StorageContext: kontekst przechowywania danych
- VectorStoreIndex: indeks wektorowy
- AgentRunner i FunctionCallingAgentWorker: komponenty do obsługi agentów
- SentenceSplitter: dzielenie tekstu na zdania
- VectorIndexRetriever: pobieranie danych z indeksu
- FunctionTool i ToolOutput: narzędzia do wywoływania funkcji

Z innych modułów llama_index:
- OpenAIEmbedding: tworzenie embedingów przez OpenAI
- OpenAI: dostęp do modeli językowych OpenAI
- LanceDBVectorStore: baza danych wektorów

Dodatkowo:
- tqdm: pasek postępu
- google.colab.userdata: dostęp do danych użytkownika w Google Colab

In [ ]:
class CFG:
  model1 = "gpt-4o-mini"
  model2 = "text-embedding-3-large"
  temperature = 0.1
  chunksize = 1024
  datadir = '/content/'

In [ ]:
# Using OpenAI API for embeddings/llms
os.environ["OPENAI_API_KEY"] = userdata.get('openaivision')


In [ ]:

llm = OpenAI(model= CFG.model1)
Settings.llm = llm

embed_model = OpenAIEmbedding(model = CFG.model2)
Settings.embed_model = embed_model

Ten fragment kodu konfiguruje dwa kluczowe elementy:

1. Konfiguracja modelu językowego (LLM):
```python
llm = OpenAI(model= CFG.model1)
Settings.llm = llm
```
Tworzona jest instancja modelu OpenAI z parametrem określonym w CFG.model1 (prawdopodobnie gpt-4 lub podobny). Ten model jest następnie ustawiany jako domyślny LLM w globalnych ustawieniach.

2. Konfiguracja modelu do embedingów:
```python
embed_model = OpenAIEmbedding(model = CFG.model2)
Settings.embed_model = embed_model
```
Tworzona jest instancja modelu do tworzenia embedingów określonego w CFG.model2. Model ten jest ustawiany jako domyślny dla embedingów w globalnych ustawieniach.

# Functions

## Helper functions

In [ ]:
def load_and_index_document_from_file(
    file_path: str, vector_store: LanceDBVectorStore
) -> VectorStoreIndex:
    """Load a document from a single file and index it."""
    with open(file_path, "r") as f:
        data = json.load(f)
        document = Document(text=json.dumps(data))

    parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
    nodes = parser.get_nodes_from_documents([document])
    storage_context = StorageContext.from_defaults(vector_store=vector_store)
    return VectorStoreIndex(nodes, storage_context=storage_context)



Ta funkcja służy do załadowania i zaindeksowania dokumentu. Przyjmuje dwa parametry:
- file_path: ścieżka do pliku
- vector_store: instancja magazynu wektorów LanceDB

Oto co robi krok po kroku:

1. Otwiera plik JSON i ładuje jego zawartość:
```python
with open(file_path, "r") as f:
    data = json.load(f)
```

2. Tworzy obiekt Document, przekształcając dane z powrotem do formatu JSON:
```python
document = Document(text=json.dumps(data))
```

3. Tworzy parser tekstu, który dzieli tekst na mniejsze części:
```python
parser = SentenceSplitter(chunk_size=1024, chunk_overlap=200)
```
- chunk_size=1024 oznacza maksymalny rozmiar fragmentu
- chunk_overlap=200 oznacza że fragmenty nachodzą na siebie o 200 znaków

4. Dzieli dokument na węzły:
```python
nodes = parser.get_nodes_from_documents([document])
```

5. Tworzy kontekst przechowywania z użyciem dostarczonego vector_store:
```python
storage_context = StorageContext.from_defaults(vector_store=vector_store)
```

6. Zwraca zaindeksowany dokument jako VectorStoreIndex:
```python
return VectorStoreIndex(nodes, storage_context=storage_context)
```

Funkcja ta jest używana do przygotowania dokumentów do wyszukiwania semantycznego - dzieli tekst na mniejsze części i tworzy ich reprezentacje wektorowe.

In [ ]:
def create_retriever(index: VectorStoreIndex) -> VectorIndexRetriever:
    """Create a retriever from the index."""
    return index.as_retriever(similarity_top_k=5)

Ta funkcja tworzy obiekt służący do wyszukiwania w indeksie wektorowym. Jest bardzo prosta, ale pełni ważną rolę:

1. Przyjmuje jeden parametr - index typu VectorStoreIndex, czyli wcześniej utworzony indeks dokumentów

2. Tworzy i zwraca retriever (narzędzie do wyszukiwania) poprzez wywołanie:
```python
index.as_retriever(similarity_top_k=5)
```

Parametr similarity_top_k=5 oznacza, że przy każdym wyszukiwaniu zwracanych będzie 5 najbardziej podobnych fragmentów tekstu.

Ten retriever będzie używany do znajdowania fragmentów dokumentów najbardziej odpowiadających zadanemu zapytaniu na podstawie podobieństwa wektorów.

# Data

In [ ]:
# Vector store setup
problems_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="problems_table",
    mode="overwrite",
)

parts_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="parts_table",
    mode="overwrite",
)

diagnostics_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="diagnostics_table",
    mode="overwrite",
)

cost_estimates_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="cost_estimates_table",
    mode="overwrite",
)

maintenance_schedules_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="maintenance_schedules_table",
    mode="overwrite",
)

cars_vector_store = LanceDBVectorStore(
    uri="./lancedb",
    table_name="car_maintenance_table",
    mode="overwrite",
)

Ten fragment kodu tworzy sześć magazynów wektorów w bazie danych LanceDB dla różnych typów danych związanych z serwisem samochodowym:

1. problems_vector_store - do przechowywania problemów
2. parts_vector_store - do przechowywania części
3. diagnostics_vector_store - do przechowywania diagnostyk
4. cost_estimates_vector_store - do przechowywania wycen
5. maintenance_schedules_vector_store - do przechowywania harmonogramów konserwacji
6. cars_vector_store - do przechowywania informacji o samochodach

Każdy magazyn jest konfigurowany z następującymi parametrami:
- uri="./lancedb" - ścieżka do lokalnej bazy danych
- table_name - unikalna nazwa tabeli dla każdego typu danych
- mode="overwrite" - tryb nadpisywania istniejących danych

Ta struktura pozwala na oddzielne przechowywanie i wyszukiwanie różnych typów informacji związanych z serwisem samochodowym.

In [ ]:
# Load and index documents directly from file paths
problems_index = load_and_index_document_from_file( CFG.datadir + "problems.json", problems_vector_store)

parts_index = load_and_index_document_from_file(
    CFG.datadir + "parts.json", parts_vector_store
)
cars_index = load_and_index_document_from_file(
    CFG.datadir + "cars_models.json", cars_vector_store
)
diagnostics_index = load_and_index_document_from_file(
    CFG.datadir + "diagnostics.json",
    diagnostics_vector_store,
)
cost_estimates_index = load_and_index_document_from_file(
    CFG.datadir + "cost_estimates.json",
    cost_estimates_vector_store,
)
maintenance_schedules_index = load_and_index_document_from_file(
    CFG.datadir + "maintenance.json",
    maintenance_schedules_vector_store,
)


Ten fragment kodu ładuje i indeksuje dokumenty JSON do wcześniej utworzonych magazynów wektorowych:

1. problems_index - indeksuje dane o problemach z pliku "problems.json"
2. parts_index - indeksuje dane o częściach z "parts.json"
3. cars_index - indeksuje dane o modelach samochodów z "cars_models.json"
4. diagnostics_index - indeksuje dane diagnostyczne z "diagnostics.json"
5. cost_estimates_index - indeksuje dane o kosztach z "cost_estimates.json"
6. maintenance_schedules_index - indeksuje harmonogramy konserwacji z "maintenance.json"

Dla każdego typu danych:
- Ścieżka do pliku jest tworzona przez połączenie CFG.datadir z nazwą pliku
- Używana jest wcześniej zdefiniowana funkcja load_and_index_document_from_file()
- Każdy dokument jest ładowany do odpowiadającego mu magazynu wektorów

Te zaindeksowane dokumenty będą służyć jako baza wiedzy, w której można wyszukiwać informacje za pomocą zapytań semantycznych.

In [ ]:

problems_retriever = create_retriever(problems_index)
parts_retriever = create_retriever(parts_index)
cars_retriever = create_retriever(cars_index)
diagnostics_retriever = create_retriever(diagnostics_index)
cost_estimates_retriever = create_retriever(cost_estimates_index)
maintenance_schedules_retriever = create_retriever(maintenance_schedules_index)

Ten fragment tworzy narzędzia do wyszukiwania (retrievery) dla każdego z wcześniej utworzonych indeksów:

1. problems_retriever - do wyszukiwania w danych o problemach
2. parts_retriever - do wyszukiwania informacji o częściach
3. cars_retriever - do wyszukiwania danych o samochodach
4. diagnostics_retriever - do wyszukiwania informacji diagnostycznych
5. cost_estimates_retriever - do wyszukiwania wycen
6. maintenance_schedules_retriever - do wyszukiwania harmonogramów konserwacji

Każdy retriever jest tworzony za pomocą wcześniej zdefiniowanej funkcji create_retriever(), która ustawia limit 5 najbardziej podobnych wyników dla każdego wyszukiwania.

Te retrievery będą używane do znajdowania odpowiednich informacji w bazie wiedzy na podstawie zapytań użytkownika.

In [ ]:
# test retriever
query = "My brake pad isn't working , what's the cost for the solution?"
query_engine = cost_estimates_index.as_query_engine()
response = query_engine.query(query)
results = cost_estimates_retriever.retrieve(query)

# Print the response summary
print(f"Response: {response}")

# Print only relevant information from results
for result in results:
    print(f"Result - Node ID: {result.node_id}")
    print(f"Relevant Text: {result.text[:150]}...")
    print(f"Score: {result.score:.3f}")

Response: The cost for brake pad replacement typically averages around $150, with a range between $100 and $300.
Result - Node ID: 0c7330a8-6a68-48e1-b9d0-a014c000dad1
Relevant Text: [{"repair": "Brake pad replacement", "average_cost": 150, "cost_range": {"min": 100, "max": 300}}, {"repair": "Oil change", "average_cost": 50, "cost_...
Score: 0.301
Result - Node ID: 617dac35-47f2-46cf-8a35-818fa3f05eb6
Relevant Text: "max": 600}}, {"repair": "Fuel Pump Replacement", "average_cost": 500, "cost_range": {"min": 400, "max": 700}}, {"repair": "AC Compressor Replacement"...
Score: 0.280


# Agent setup

In [ ]:
def retrieve_problems(query: str) -> str:
    """Searches the problem catalog to find relevant automotive problems for the query."""
    docs = problems_retriever.retrieve(query)
    information = str([doc.text[:200] for doc in docs])
    return information


def retrieve_parts(query: str) -> str:
    """Searches the parts catalog to find relevant parts for the query."""
    docs = parts_retriever.retrieve(query)
    information = str([doc.text[:200] for doc in docs])
    return information


def retrieve_car_details(make: str, model: str, year: int) -> str:
    """Retrieves the make, model, and year of the car."""
    docs = car_details_retriever.retrieve(make, model, year)
    information = str([doc.text[:200] for doc in docs])


def diagnose_car_problem(symptoms: str) -> str:
    """Uses the diagnostics database to find potential causes for given symptoms."""
    docs = diagnostics_retriever.retrieve(symptoms)
    information = str([doc.text[:200] for doc in docs])
    return information


def estimate_repair_cost(problem: str) -> str:
    """Provides a cost estimate for a given car problem or repair."""
    docs = cost_estimates_retriever.retrieve(problem)
    information = str([doc.text[:200] for doc in docs])
    return information


def get_maintenance_schedule(mileage: int) -> str:
    """Retrieves the recommended maintenance schedule based on mileage."""
    docs = maintenance_schedules_retriever.retrieve(str(mileage))
    information = str([doc.text[:200] for doc in docs])
    return information

To podstawowe funkcje do wyszukiwania informacji w bazie wiedzy o samochodach:

1. retrieve_problems - szuka informacji o problemach samochodowych na podstawie zapytania tekstowego. Zwraca pierwsze 200 znaków z każdego znalezionego dokumentu.

2. retrieve_parts - wyszukuje części samochodowe pasujące do zapytania. Podobnie zwraca pierwsze 200 znaków z każdego dopasowanego dokumentu.

3. retrieve_car_details - pobiera informacje o konkretnym samochodzie na podstawie marki, modelu i roku. Uwaga: brakuje tu return statement.

4. diagnose_car_problem - na podstawie opisanych symptomów szuka potencjalnych przyczyn problemu w bazie diagnostycznej.

5. estimate_repair_cost - szacuje koszty naprawy na podstawie opisu problemu.

6. get_maintenance_schedule - znajduje zalecany harmonogram konserwacji dla danego przebiegu.

Wszystkie te funkcje mają podobną strukturę:
- przyjmują parametr wyszukiwania
- używają odpowiedniego retrievera do znalezienia pasujących dokumentów
- zwracają pierwsze 200 znaków z każdego znalezionego dokumentu jako string

In [ ]:
def comprehensive_diagnosis(symptoms: str) -> str:
    """
    Provides a comprehensive diagnosis including possible causes, estimated costs, and required parts.

    Args:
        symptoms: A string describing the car's symptoms.

    Returns:
        A string with a comprehensive diagnosis report.
    """
    # Use existing tools
    possible_causes = diagnose_car_problem(symptoms)

    # Extract the most likely cause (this is a simplification)
    likely_cause = possible_causes[0] if possible_causes else "Unknown issue"

    estimated_cost = estimate_repair_cost(likely_cause)
    required_parts = retrieve_parts(likely_cause)

    report = f"Comprehensive Diagnosis Report:\n\n"
    report += f"Symptoms: {symptoms}\n\n"
    report += f"Possible Causes:\n{possible_causes}\n\n"
    report += f"Most Likely Cause: {likely_cause}\n\n"
    report += f"Estimated Cost:\n{estimated_cost}\n\n"
    report += f"Required Parts:\n{required_parts}\n\n"
    report += "Please note that this is an initial diagnosis. For accurate results, please consult with our professional mechanic."

    return report


def get_car_model_info(
    mileage: int, car_make: str, car_model: str, car_year: int
) -> dict:
    """Retrieve car model information from cars_models.json."""
    with open(CFG.datadir + "cars_models.json", "r") as file:
        car_models = json.load(file)

    for car in car_models:
        if (
            car["car_make"].lower() == car_make.lower()
            and car["car_model"].lower() == car_model.lower()
            and car["car_year"] == car_year
        ):
            return car
    return {}


def retrieve_car_details(make: str, model: str, year: int) -> str:
    """Retrieves the make, model, and year of the car and return the common issues if any."""
    car_details = get_car_model_info(
        0, make, model, year
    )  # Using 0 for mileage to get general details
    if car_details:
        return f"{year} {make} {model} - Common Issues: {', '.join(car_details['common_issues'])}"
    return f"{year} {make} {model} - No common issues found."


Przeanalizujmy te trzy funkcje:

1. comprehensive_diagnosis - tworzy pełny raport diagnostyczny:
   - przyjmuje opis symptomów
   - znajduje możliwe przyczyny używając diagnose_car_problem
   - wybiera najbardziej prawdopodobną przyczynę (pierwszy wynik)
   - szacuje koszty naprawy dla tej przyczyny
   - znajduje potrzebne części
   - tworzy sformatowany raport zawierający wszystkie te informacje
   - dodaje zastrzeżenie o konieczności konsultacji z mechanikiem

2. get_car_model_info - pobiera szczegółowe informacje o samochodzie z pliku JSON:
   - otwiera plik cars_models.json
   - szuka samochodu o pasującej marce, modelu i roku
   - porównuje wartości ignorując wielkość liter dla marki i modelu
   - zwraca wszystkie informacje o znalezionym samochodzie lub pusty słownik

3. retrieve_car_details - znajduje typowe problemy dla danego samochodu:
   - wykorzystuje get_car_model_info (z przebiegiem 0 dla ogólnych informacji)
   - jeśli znajdzie informacje, zwraca tekst z marką, modelem, rokiem i listą typowych problemów
   - jeśli nie znajdzie informacji, zwraca tekst o braku znanych problemów

In [ ]:
def plan_maintenance(mileage: int, car_make: str, car_model: str, car_year: int) -> str:
    """
    Creates a comprehensive maintenance plan based on the car's mileage and details.

    Args:
        mileage: The current mileage of the car.
        car_make: The make of the car.
        car_model: The model of the car.
        car_year: The year the car was manufactured.

    Returns:
        A string with a comprehensive maintenance plan.
    """
    car_details = retrieve_car_details(car_make, car_model, car_year)
    car_model_info = get_car_model_info(mileage, car_make, car_model, car_year)

    plan = f"Maintenance Plan for {car_year} {car_make} {car_model} at {mileage} miles:\n\n"
    plan += f"Car Details: {car_details}\n\n"

    if car_model_info:
        plan += f"Common Issues:\n"
        for issue in car_model_info["common_issues"]:
            plan += f"- {issue}\n"

        plan += f"\nEstimated Time: {car_model_info['estimated_time']}\n\n"
    else:
        plan += (
            "No specific maintenance tasks found for this car model and mileage.\n\n"
        )

    plan += "Please consult with our certified mechanic for a more personalized maintenance plan."

    return plan


def create_calendar_invite(
    event_type: str, car_details: str, duration: int = 60
) -> str:
    """
    Simulates creating a calendar invite for a car maintenance or repair event.

    Args:
        event_type: The type of event (e.g., "Oil Change", "Brake Inspection").
        car_details: Details of the car (make, model, year).
        duration: Duration of the event in minutes (default is 60).

    Returns:
        A string describing the calendar invite.
    """
    # Simulate scheduling the event for next week
    event_date = datetime.now() + timedelta(days=7)
    event_time = event_date.replace(hour=10, minute=0, second=0, microsecond=0)

    invite = f"Calendar Invite Created:\n\n"
    invite += f"Event: {event_type} for {car_details}\n"
    invite += f"Date: {event_time.strftime('%Y-%m-%d')}\n"
    invite += f"Time: {event_time.strftime('%I:%M %p')}\n"
    invite += f"Duration: {duration} minutes\n"
    invite += f"Location: Your Trusted Auto Shop, 123 Main St, Bengaluru, India\n\n"

    return invite


def coordinate_car_care(
    query: str, car_make: str, car_model: str, car_year: int, mileage: int
) -> str:
    """
    Coordinates overall car care by integrating diagnosis, maintenance planning, and scheduling.

    Args:
        query: The user's query or description of the issue.
        car_make: The make of the car.
        car_model: The model of the car.
        car_year: The year the car was manufactured.
        mileage: The current mileage of the car.

    Returns:
        A string with a comprehensive car care plan.
    """
    car_details = retrieve_car_details(car_make, car_model, car_year)

    # Check if it's a problem or routine maintenance
    if "problem" in query.lower() or "issue" in query.lower():
        diagnosis = comprehensive_diagnosis(query)
        plan = f"Based on your query, here's a diagnosis:\n\n{diagnosis}\n\n"

        # Extract the most likely cause (this is a simplification)
        likely_cause = diagnosis.split("Most Likely Cause:")[1].split("\n")[0].strip()

        # Create a calendar invite for repair
        invite = create_calendar_invite(f"Repair: {likely_cause}", car_details)
        plan += f"I've prepared a calendar invite for the repair:\n\n{invite}\n\n"
    else:
        maintenance_plan = plan_maintenance(mileage, car_make, car_model, car_year)
        plan = f"Here's your maintenance plan:\n\n{maintenance_plan}\n\n"

        # Create a calendar invite for the next maintenance task
        next_task = maintenance_plan.split("Task:")[1].split("\n")[0].strip()
        invite = create_calendar_invite(f"Maintenance: {next_task}", car_details)
        plan += f"I've prepared a calendar invite for your next maintenance task:\n\n{invite}\n\n"

    plan += "Remember to consult with a professional mechanic for personalized advice and service."

    return plan



Te trzy funkcje stanowią zaawansowaną część systemu zarządzania serwisem samochodowym:

1. plan_maintenance - tworzy plan konserwacji:
   - pobiera szczegóły samochodu i informacje o modelu
   - tworzy nagłówek planu z danymi samochodu i przebiegiem
   - jeśli znajdzie informacje o modelu:
     - wypisuje listę typowych problemów
     - dodaje szacowany czas
   - jeśli nie znajdzie informacji, informuje o braku specyficznych zadań
   - dodaje zalecenie konsultacji z mechanikiem

2. create_calendar_invite - tworzy zaproszenie na wizytę:
   - ustawia datę na tydzień do przodu
   - ustawia godzinę na 10:00
   - tworzy zaproszenie zawierające:
     - typ wydarzenia
     - szczegóły samochodu
     - datę i godzinę
     - czas trwania (domyślnie 60 minut)
     - lokalizację warsztatu

3. coordinate_car_care - główna funkcja koordynująca:
   - sprawdza czy zapytanie dotyczy problemu czy rutynowej konserwacji
   - dla problemu:
     - wykonuje diagnostykę
     - wyciąga najbardziej prawdopodobną przyczynę
     - tworzy zaproszenie na naprawę
   - dla rutynowej konserwacji:
     - tworzy plan konserwacji
     - wyciąga następne zadanie
     - tworzy zaproszenie na konserwację
   - dodaje przypomnienie o konsultacji z mechanikiem

Funkcje te współpracują ze sobą, tworząc kompletny system od diagnozy przez planowanie po umówienie wizyty.

In [ ]:
retrieve_problems_tool = FunctionTool.from_defaults(fn=retrieve_problems)
retrieve_parts_tool = FunctionTool.from_defaults(fn=retrieve_parts)
diagnostic_tool = FunctionTool.from_defaults(fn=diagnose_car_problem)
cost_estimator_tool = FunctionTool.from_defaults(fn=estimate_repair_cost)
maintenance_schedule_tool = FunctionTool.from_defaults(fn=get_maintenance_schedule)
comprehensive_diagnostic_tool = FunctionTool.from_defaults(fn=comprehensive_diagnosis)
maintenance_planner_tool = FunctionTool.from_defaults(fn=plan_maintenance)
calendar_invite_tool = FunctionTool.from_defaults(fn=create_calendar_invite)
car_care_coordinator_tool = FunctionTool.from_defaults(fn=coordinate_car_care)
retrieve_car_details_tool = FunctionTool.from_defaults(fn=retrieve_car_details)


Te linie kodu zamieniają wcześniej zdefiniowane funkcje na narzędzia (tools) do użycia z agentem:

1. retrieve_problems_tool - narzędzie do wyszukiwania problemów
2. retrieve_parts_tool - narzędzie do wyszukiwania części
3. diagnostic_tool - narzędzie do diagnostyki
4. cost_estimator_tool - narzędzie do szacowania kosztów
5. maintenance_schedule_tool - narzędzie do harmonogramów konserwacji
6. comprehensive_diagnostic_tool - narzędzie do pełnej diagnostyki
7. maintenance_planner_tool - narzędzie do planowania konserwacji
8. calendar_invite_tool - narzędzie do tworzenia zaproszeń kalendarzowych
9. car_care_coordinator_tool - narzędzie do koordynacji opieki nad samochodem
10. retrieve_car_details_tool - narzędzie do pobierania szczegółów samochodu

Każde narzędzie jest tworzone metodą FunctionTool.from_defaults(), która zamienia zwykłą funkcję Pythona na narzędzie, które może być używane przez agenta do wykonywania zadań.

In [ ]:

tools = [
    retrieve_problems_tool,
    retrieve_parts_tool,
    diagnostic_tool,
    cost_estimator_tool,
    maintenance_schedule_tool,
    comprehensive_diagnostic_tool,
    maintenance_planner_tool,
    calendar_invite_tool,
    car_care_coordinator_tool,
    retrieve_car_details_tool,
]




In [ ]:

agent_worker = FunctionCallingAgentWorker.from_tools(tools, llm=llm, verbose=True)
agent = AgentRunner(agent_worker)


Te dwie linie kodu konfigurują system agenta:

1. Tworzenie workera agenta:
```python
agent_worker = FunctionCallingAgentWorker.from_tools(tools, llm=llm, verbose=True)
```
- używa wcześniej zdefiniowanej listy narzędzi (tools)
- wykorzystuje skonfigurowany model językowy (llm)
- verbose=True włącza szczegółowe logowanie działań

2. Tworzenie agenta:
```python
agent = AgentRunner(agent_worker)
```
- tworzy głównego agenta wykorzystującego przygotowanego workera
- ten agent będzie odpowiedzialny za obsługę zapytań użytkownika i koordynację użycia odpowiednich narzędzi

Ten system agenta będzie mógł analizować zapytania, wybierać odpowiednie narzędzia i wykonywać zadania związane z serwisem samochodowym w sposób zautomatyzowany.

# Run

In [ ]:
response = agent.chat(
    "My car has 60,000 miles on it. What maintenance should I be doing now, and how much will it cost?"
)

Added user message to memory: My car has 60,000 miles on it. What maintenance should I be doing now, and how much will it cost?
=== LLM Response ===
To provide you with the best maintenance plan and cost estimate, I need to know the make, model, and year of your car. Could you please provide that information?


In [ ]:
response = agent.chat(
    "I have a honda accord of 2017 model and it's mileage is 30000 right now, what are some common issues?"
)

Added user message to memory: I have a honda accord of 2017 model and it's mileage is 30000 right now, what are some common issues?
=== Calling Function ===
Calling function: retrieve_car_details with args: {"make": "Honda", "model": "Accord", "year": 2017}
=== Function Output ===
2017 Honda Accord - Common Issues: Engine misfires, Electrical issues
=== LLM Response ===
The common issues for a 2017 Honda Accord include:

1. **Engine Misfires**: This can be caused by faulty spark plugs, ignition coils, or fuel injectors.
2. **Electrical Issues**: Problems with the electrical system, including battery, alternator, or wiring.

If you have any specific symptoms or concerns, please let me know, and I can assist you further!


In [ ]:
response = agent.chat(
    "Can you help me with these issues, I want to do some maintenance and what's the cost for all of this services? for parts and all which will be required"
)

Added user message to memory: Can you help me with these issues, I want to do some maintenance and what's the cost for all of this services? for parts and all which will be required
=== Calling Function ===
Calling function: plan_maintenance with args: {"mileage": 30000, "car_make": "Honda", "car_model": "Accord", "car_year": 2017}
=== Function Output ===
Maintenance Plan for 2017 Honda Accord at 30000 miles:

Car Details: 2017 Honda Accord - Common Issues: Engine misfires, Electrical issues

Common Issues:
- Engine misfires
- Electrical issues

Estimated Time: 3-4 hours

Please consult with our certified mechanic for a more personalized maintenance plan.
=== Calling Function ===
Calling function: estimate_repair_cost with args: {"problem": "Engine misfires"}
=== Function Output ===
['"max": 600}}, {"repair": "Fuel Pump Replacement", "average_cost": 500, "cost_range": {"min": 400, "max": 700}}, {"repair": "AC Compressor Replacement", "average_cost": 800, "cost_range": {"min": 600, ', '